
# The Best Model As Of July 23 2020

10 jets, #nofilter, PtEtaPhi network, restricted dataset

Create and train network

In [1]:
import os
import sys
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="6"
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers import Dense, Conv2D, Flatten, Dropout
sys.path.append(os.path.realpath("../../"))
import ptetaphi_nn
import tools

# get data file path
with open("/home/cmccracken/start_tf/bbb/filepath.txt", 'r') as f:
    filename = f.read()
    
s_table = tools.open_file(filename, sort_by="tag", pt_cut=-40)  # negative means pt < 40

# filter for events with 3 b tags
nt3 = s_table.nbtags==3 
events = s_table[nt3]
print(len(events))

cutoff = 10  # not many events have >10 jets
# "pad" = ensure all events have same length, cut off ends if needed
events = tools.pad(events, cutoff)

# make and train network
nn = ptetaphi_nn.PtEtaPhiNN(events, chop=0, print_summary=True)
nn.learn(epochs=800)

Using TensorFlow backend.


777150 total events found
777150 events left after pt cut
sorting data by tag
212
creating default model
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 30)                930       
_________________________________________________________________
dense_2 (Dense)              (None, 700)               21700     
_________________________________________________________________
dropout_1 (Dropout)          (None, 700)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               350500    
_________________________________________________________________
dropout_2 (Dropout)          (None, 500)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 300)               150300    
_______________

ValueError: Error when checking input: expected dense_1_input to have shape (30,) but got array with shape (27,)

Evaluate network

In [4]:
nn.evaluate(savename="low_cut")

ValueError: Error when checking input: expected dense_8_input to have shape (30,) but got array with shape (27,)

Save model for use later

In [ ]:
# save the architecture string to a file
models_dir = "/home/cmccracken/start_tf/bbb/models/"
with open(models_dir+'architecture_30_07_2020.json', 'w') as arch_file:
    arch_file.write(nn.model.to_json())
# now save the weights as an HDF5 file
nn.model.save_weights(models_dir+'weights_30_07_2020.h5')
# use nn_tester to get csv!